# Plot stats of dv/v model parameters with MCMC.

2023.4.4 Kurama Okubo

Plot MCMC stats of model paramters and AIC/BIC.

- 2023/4/19 Update for the case with fixed model parameters
- 2023/10/6 update figure notations

## Plot list
1. All model parameter with base and and wlin model.
2. The comparison of AIC and BIC between the models.
3. Model parameter of blin; plot as a function of fault normal distance.

## Removal of FROB
Some of stations in BP network has an potentially intrumental or environmental noise at 1.0Hz and some other frequencies. `BP.FROB` shows it in all 3 components during almost half of the analyzing period (2012-2022). 

<img src="PSD_BP.FROB.40.SP1.png" alt="fig_FROB" width="400"/>
Figure. Example of 1 day PSD at BP.FROB vertical component.

It has an effect more on the auto-correlation as the periodic noise causes strong coherence to the other frequency components. It is mitigated with the cross-correlation as the periodic noise is not always coherent to the stations without it. The channel weighted dv/v also decrease the bias due to the low cc to the stacked reference (**need to be checked**).

In this notebook and following plots on the model parameters, we decided to exclude the data including FROB, as the dv/v cannot avoid the effect from the periodic noise in the statistical analysis.


In [ ]:
# %load_ext autoreload
# %autoreload 2

import datetime
import os
import time

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import matplotlib.dates as mdates
%matplotlib inline
from IPython.display import display, Math

import numpy as np
import pandas as pd
import h5py
import pickle

import shutil
from tqdm import tqdm
import glob

import matplotlib as mpl
import seaborn as sns 

# import mcmc functions
from scattermatrix import *
from MCMC_func import *


plt.rcParams["font.family"] = 'Arial'
# plt.rcParams["font.sans-serif"] = "DejaVu Sans, Arial, Helvetica, Lucida Grande, Verdana, Geneva, Lucid, Avant Garde, sans-serif"
plt.rcParams["font.size"] = 12
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 4.75
plt.rcParams["xtick.major.width"] = 0.75
plt.rcParams["xtick.minor.size"] = 3
plt.rcParams["xtick.minor.width"] = 0.4
plt.rcParams["xtick.minor.visible"] = True

plt.rcParams["ytick.direction"] = "in"
plt.rcParams["ytick.major.size"] = 4.75
plt.rcParams["ytick.major.width"] = 0.75
plt.rcParams["ytick.minor.size"] = 3
plt.rcParams["ytick.minor.width"] = 0.4
plt.rcParams["ytick.minor.visible"] = True

plt.rcParams["savefig.transparent"] = True

plt.rcParams['axes.linewidth'] = 0.75


In [ ]:
inputdatadir = "../data_stats"
freqband = "0.9-1.2"

residu_absmean_thresh = 0.05
residu_var_thresh = 0.002

js = 4 # jitter marker size

output_imgdir = "../figure_stats"


In [ ]:
# # read table
df_all = pd.DataFrame()
df_all = pd.concat([df_all, pd.read_csv(inputdatadir+"/MCMC_modelparam_all_{}_{}_{}.csv".format("stretching", freqband, "base"), index_col=0)])
df_all = pd.concat([df_all, pd.read_csv(inputdatadir+"/MCMC_modelparam_all_{}_{}_{}.csv".format("stretching", freqband, "wlin"), index_col=0)])
df_all = pd.concat([df_all, pd.read_csv(inputdatadir+"/MCMC_modelparam_all_{}_{}_{}.csv".format("mwcs", freqband, "base"), index_col=0)])
df_all = pd.concat([df_all, pd.read_csv(inputdatadir+"/MCMC_modelparam_all_{}_{}_{}.csv".format("mwcs", freqband, "wlin"), index_col=0)])

In [ ]:
# save the all parameters in a single file for publication
df_all.to_csv(inputdatadir+"/MCMC_modelparam_all_allcases_master.csv", float_format='%12.10f')

In [ ]:
# Read the model parameter from the compiled csv
df_all_2 = pd.read_csv(inputdatadir+"/MCMC_modelparam_all_allcases_master.csv", index_col=0)

print(f"The compiled CSV is identical to the original: {np.all(np.abs((df_all.iloc[:,3:] - df_all_2.iloc[:,3:]).fillna(0)) < 1e-10)}")
df_all = df_all_2

In [ ]:
# Threshold out the data with large variance of residuals between data and model
df_all_residuthresh = df_all[df_all.residu_var<residu_var_thresh]

# Remove the data associated with auto- and cross-correlation with BP."FROB" due to the prominent periodic noise 
df_all_residuthresh = df_all_residuthresh[["FROB" not in x for x in df_all_residuthresh.index]]

In [ ]:
cpalette=np.array(sns.color_palette("Set2"))[1:5]
cpalette

In [ ]:
# Convert the units associated with log time and b_lin
df_all_residuthresh.loc[:, "tmin1"] = df_all_residuthresh["log10tmin1"].apply(lambda x: 10**x/86400/365)
df_all_residuthresh.loc[:, "tmin2"] = df_all_residuthresh["log10tmin2"].apply(lambda x: 10**x/86400/365)
df_all_residuthresh.loc[:, "tmax1"] = df_all_residuthresh["log10tmax1"].apply(lambda x: 10**x/86400/365)
df_all_residuthresh.loc[:, "tmax2"] = df_all_residuthresh["log10tmax2"].apply(lambda x: 10**x/86400/365)
df_all_residuthresh.loc[:, "blin_1overyear"] = df_all_residuthresh["b_{lin}"].apply(lambda x: x*365)


In [ ]:
# rename modelcase
df_all_residuthresh.loc[:, "modelcase"] = df_all_residuthresh["modelcase"].str.replace('wlin','w. linear trend')          
df_all_residuthresh.rename(columns={"dvvmethod": "method"}, inplace = True)

In [ ]:
# convert mwcs to uppercase
df_all_residuthresh.loc[df_all_residuthresh["method"]=="mwcs", "method"] = "MWCS"

In [ ]:
df_all_residuthresh

# Plot AIC and BIC

In [ ]:
#1. Plot absolute AIC and BIC

np.random.seed(20230420)
fig, axs = plt.subplots(1, 2, figsize=(9, 4))

key="AIC"
sns.boxplot(x="modelcase", y=key, hue="method",data=df_all_residuthresh, showfliers=False, palette=cpalette, ax=axs[0])
sns.stripplot(x="modelcase", y=key, hue="method", data=df_all_residuthresh, jitter=True, dodge=True, palette='dark:k', legend=None, size=js, ax=axs[0])
axs[0].set_ylabel("AIC")
axs[0].set_ylim([-8500, -5500])

key="BIC"
sns.boxplot(x="modelcase", y=key, hue="method",data=df_all_residuthresh, showfliers=False, palette=cpalette, ax=axs[1])
sns.stripplot(x="modelcase", y=key, hue="method", data=df_all_residuthresh, jitter=True, dodge=True, palette='dark:k', legend=None, size=js, ax=axs[1])
axs[1].set_ylabel("BIC")
axs[1].set_ylim([-8500, -5500])

plt.setp(axs, xlabel=None)
axs[0].get_legend().remove()
sns.move_legend(axs[1], "upper left", bbox_to_anchor=(1, 1), )
sns.despine()
# axs.set(axs, xlabel=None)

fig.tight_layout()

foname = (output_imgdir+"/MCMC_absoluteAICBIC_{}.png".format(freqband))
plt.savefig(foname, dpi=300, bbox_inches='tight')


**NOTE** The station pair associated with the large AIC shown in stretching with linear trend is thresholded out for the case with the base model. Therefore, $\Delta AIC$ and $\Delta BIC$ is not defined. See below the search of station pairs.

In [ ]:
df_all_residuthresh.sort_values(by="AIC", ascending=False).head()

In [ ]:
print(df_all_residuthresh.loc["BP.EADB-BP.EADB", :].shape)
df_all_residuthresh.loc["BP.EADB-BP.EADB", :]

In [ ]:
df_all_residuthresh.loc["BP.JCNB-BP.LCCB", :].shape

### Compute ΔAIC and ΔBIC for each station pair
The comparison of median with the abslute AIC and BIC is essentially not appropriate. We need to compair them by station pairs. They are defined as

$$\Delta AIC = AIC_{w.lin} - AIC_{base},$$
$$\Delta BIC = BIC_{w.lin} - BIC_{base}.$$

If they are negative, i.e. IC with linear trend is smaller, the model with linear trend is considered as better to the data.
There could be a lack in the pair between base and w. linear trend models due to the thresholding of residuals. Thus, we first search if the pair exists, and compute the difference between them.

In [ ]:
stationlist = np.sort(np.unique(df_all_residuthresh.index.values))

In [ ]:
df_delAICBIC = pd.DataFrame(columns=["method", "ICtype", "value"])

for stindex in stationlist:
# stindex = stationlist[10]
#     print(stindex)
    df_pair = df_all_residuthresh.loc[stindex, :]

    # stretching
    if not isinstance(df_pair["method"]=="stretching", (bool)):
        df_pair_stretching = df_pair[df_pair["method"]=="stretching"]
        if len(df_pair_stretching)==2:
            delAIC_wlin = df_pair_stretching[df_pair_stretching["modelcase"]=="w. linear trend"].AIC.values[0]
            delAIC_base = df_pair_stretching[df_pair_stretching["modelcase"]=="base"].AIC.values[0]
            delBIC_wlin = df_pair_stretching[df_pair_stretching["modelcase"]=="w. linear trend"].BIC.values[0]
            delBIC_base = df_pair_stretching[df_pair_stretching["modelcase"]=="base"].BIC.values[0]
            delAIC_stretching = delAIC_wlin - delAIC_base
            delBIC_stretching = delBIC_wlin - delBIC_base
        else:
            delAIC_stretching = np.nan
            delBIC_stretching = np.nan
    else:
        delAIC_stretching = np.nan
        delBIC_stretching = np.nan

#     print(delAIC_stretching, delBIC_stretching)
    
    # mwcs
    if not isinstance(df_pair["method"]=="MWCS", (bool)):
        df_pair_mwcs = df_pair[df_pair["method"]=="MWCS"]
        if len(df_pair_mwcs)==2:
            delAIC_wlin = df_pair_mwcs[df_pair_mwcs["modelcase"]=="w. linear trend"].AIC.values[0]
            delAIC_base = df_pair_mwcs[df_pair_mwcs["modelcase"]=="base"].AIC.values[0]
            delBIC_wlin = df_pair_mwcs[df_pair_mwcs["modelcase"]=="w. linear trend"].BIC.values[0]
            delBIC_base = df_pair_mwcs[df_pair_mwcs["modelcase"]=="base"].BIC.values[0]
            delAIC_mwcs = delAIC_wlin - delAIC_base
            delBIC_mwcs = delBIC_wlin - delBIC_base
        else:
            delAIC_mwcs = np.nan
            delBIC_mwcs = np.nan
    else:
        delAIC_mwcs = np.nan
        delBIC_mwcs = np.nan
        
#     print(delAIC_mwcs, delBIC_mwcs)

    ICdata = {'index':[stindex, stindex, stindex, stindex], "method":["stretching", "stretching", "MWCS", "MWCS"], "ICtype":["ΔAIC", "ΔBIC", "ΔAIC", "ΔBIC"],
              "value":[delAIC_stretching, delBIC_stretching, delAIC_mwcs, delBIC_mwcs]}

    df_delIC_pair = pd.DataFrame.from_dict(ICdata).set_index("index")
    df_delAICBIC = pd.concat([df_delAICBIC, df_delIC_pair], ignore_index=True)


In [ ]:
df_delAICBIC

In [ ]:
# cpalette_IC

In [ ]:
#1. Plot absolute AIC and BIC
cpalette_IC=np.array(sns.color_palette("Set1"))[1:5]
cpalette_IC = np.array([[141,160,203], [178,223,138]])/255
np.random.seed(20230420)
fig, ax = plt.subplots(1, 1, figsize=(4.5, 4))

# ax.grid(True)
# ax.set_axisbelow(True)

sns.boxplot(x="method", y="value", hue="ICtype", data=df_delAICBIC, showfliers=False, palette=cpalette_IC, ax=ax, saturation=0.75)
sns.stripplot(x="method", y="value", hue="ICtype", data=df_delAICBIC, jitter=True, dodge=True, palette='dark:k', size=js, legend=None, ax=ax)
ax.set_ylabel("ΔAIC, ΔBIC")
ax.set_ylim([-450, 50])

fontsize = 13 
ax.text(-0.33, 17, "ΔAIC", fontsize=fontsize)
ax.text(0.07, 17, "ΔBIC", fontsize=fontsize)

ax.text(0.67, 17, "ΔAIC", fontsize=fontsize)
ax.text(1.06, 17, "ΔBIC", fontsize=fontsize)

plt.setp(ax, xlabel=None)

ax.get_legend().remove()
# sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1), title="")
sns.despine()
# axs.set(axs, xlabel=None)
# plt.grid(axis='x')

fig.tight_layout()

foname = (output_imgdir+"/MCMC_DeltaAICBIC_{}.png".format(freqband))
plt.savefig(foname, dpi=300, bbox_inches='tight')

foname = (output_imgdir+"/MCMC_DeltaAICBIC_{}.eps".format(freqband))
plt.savefig(foname, dpi=300, bbox_inches='tight')

In [ ]:
df_all_residuthresh

In [ ]:
N_stretching = len(df_all_residuthresh[(df_all_residuthresh["method"]=="stretching") & (df_all_residuthresh["modelcase"]=="w. linear trend")])
N_mwcs = len(df_all_residuthresh[(df_all_residuthresh["method"]=="mwcs") & (df_all_residuthresh["modelcase"]=="w. linear trend")])

print(N_stretching, N_mwcs)

# Plot linear slope

In [ ]:
df_all_residuthresh_blin = df_all_residuthresh[df_all_residuthresh["modelcase"]=="w. linear trend"]

np.random.seed(20230419)

fig, ax = plt.subplots(1, 1, figsize=(4.5, 4))

key = "blin_1overyear" #"b_{lin}"

sns.boxplot(x="method", y=key, data=df_all_residuthresh_blin, showfliers=False, palette=cpalette, ax=ax, width=0.4)
sns.stripplot(x="method", y=key, data=df_all_residuthresh_blin, jitter=True, dodge=True, color='k', size=js, legend=None, ax=ax)
ax.set_ylabel(r"Slope $b_0$ [%/year]")
# ax.set_ylim([-0.002, 0.01])

plt.setp(ax, xlabel=None)
# ax.legend()
# sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1), )
sns.despine()

fig.tight_layout()

foname = (output_imgdir+"/MCMC_modelparam_boxplot_blin_{}.png".format(freqband))
plt.savefig(foname, dpi=300, bbox_inches='tight')

foname = (output_imgdir+"/MCMC_modelparam_boxplot_blin_{}.eps".format(freqband))
plt.savefig(foname, dpi=300, bbox_inches='tight')

In [ ]:
median_b_stretching = df_all_residuthresh_blin[df_all_residuthresh_blin["method"]=="stretching"][key].median()
median_b_mwcs = df_all_residuthresh_blin[df_all_residuthresh_blin["method"]=="mwcs"][key].median()
print(f"medians of the slope b are {median_b_stretching:.6f} for stretching and {median_b_mwcs:.6f} for mwcs, respectively.")

In [ ]:
df_all_residuthresh_blin[df_all_residuthresh_blin["blin_1overyear"]<0]

In [ ]:
df_all_residuthresh_blin

# Plot other model parameters

In [ ]:
from matplotlib import rc
rc('text', usetex=False)


In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(14, 6))

np.random.seed(20230420)

# a0
key="a0"
sns.boxplot(x="modelcase", y=key, hue="method",data=df_all_residuthresh, showfliers=False, palette=cpalette, ax=axs[0, 0])
sns.stripplot(x="modelcase", y=key, hue="method", data=df_all_residuthresh, jitter=True, dodge=True, palette='dark:k', legend=None, size=js, ax=axs[0, 0])
axs[0, 0].set_ylabel("$a_0$")
axs[0, 0].set_ylim([-0.1, 0.05])

# p1
key="p1"
sns.boxplot(x="modelcase", y=key, hue="method", data=df_all_residuthresh, showfliers=False, palette=cpalette, ax=axs[0, 1])
sns.stripplot(x="modelcase", y=key, hue="method", data=df_all_residuthresh, jitter=True, dodge=True, palette='dark:k', legend=None, size=js, ax=axs[0, 1])
axs[0, 1].set_ylabel("$p_1$")
axs[0, 1].set_ylim([-0.01, 0.001])
# axs[0, 1].set_yscale('log')


# p2
key="p2"
sns.boxplot(x="modelcase", y=key, hue="method", data=df_all_residuthresh, showfliers=False, palette=cpalette, ax=axs[0, 2])
sns.stripplot(x="modelcase", y=key, hue="method", data=df_all_residuthresh, jitter=True, dodge=True, palette='dark:k', legend=None, size=js, ax=axs[0, 2])
axs[0, 2].set_ylabel("$p_2$")
axs[0, 2].set_ylim([-0.0022, 0.0022])

# t_shift
key="t_{shiftdays}"
sns.boxplot(x="modelcase", y=key, hue="method", data=df_all_residuthresh, showfliers=False, palette=cpalette, ax=axs[0, 3])
sns.stripplot(x="modelcase", y=key, hue="method", data=df_all_residuthresh, jitter=True, dodge=True, palette='dark:k', legend=None, size=js, ax=axs[0, 3])
axs[0, 3].set_ylabel(r'$t_{0}^{shift}$ [day]')
axs[0, 3].set_ylim([-5, 95])
axs[0, 3].set_yticks(np.arange(0, 95, 15))

Slimit = [-0.002, 0.1]
tmaxlimit=[1e0, 1e4]

# S1
key="S1"
sns.boxplot(x="modelcase", y=key, hue="method", data=df_all_residuthresh, showfliers=False, palette=cpalette, ax=axs[1, 0])
sns.stripplot(x="modelcase", y=key, hue="method", data=df_all_residuthresh, jitter=True, dodge=True, palette='dark:k', legend=None, size=js, ax=axs[1, 0])
axs[1, 0].set_ylabel("$s_1$")
axs[1, 0].set_ylim(Slimit)


# tmax1
key="tmax1"
sns.boxplot(x="modelcase", y=key, hue="method", data=df_all_residuthresh, showfliers=False, palette=cpalette, ax=axs[1, 1])
sns.stripplot(x="modelcase", y=key, hue="method", data=df_all_residuthresh, jitter=True, dodge=True, palette='dark:k', legend=None, size=js, ax=axs[1, 1])
axs[1, 1].set_ylabel(r"$\tau_{1}^{max}$ [year]")
axs[1, 1].set_yscale('log')
axs[1, 1].set_ylim(tmaxlimit)

# S2
key="S2"
sns.boxplot(x="modelcase", y=key, hue="method", data=df_all_residuthresh, showfliers=False, palette=cpalette, ax=axs[1, 2])
sns.stripplot(x="modelcase", y=key, hue="method", data=df_all_residuthresh, jitter=True, dodge=True, palette='dark:k', legend=None, size=js, ax=axs[1, 2])
axs[1, 2].set_ylabel("$s_2$")
axs[1, 2].set_ylim(Slimit)


# tmax2
key="tmax2"
sns.boxplot(x="modelcase", y=key, hue="method", data=df_all_residuthresh, showfliers=False, palette=cpalette, ax=axs[1, 3])
sns.stripplot(x="modelcase", y=key, hue="method", data=df_all_residuthresh, jitter=True, dodge=True, palette='dark:k', legend=None, size=js, ax=axs[1, 3])
axs[1, 3].set_ylabel(r"$\tau_{2}^{max}$ [year]")
axs[1, 3].set_yscale('log')
axs[1, 3].set_ylim(tmaxlimit)

plt.setp(axs, xlabel=None)

for i in range(2):
    for j in range(4):
#         axs[i, j].grid(True, which='major', axis='y')
#         axs[i, j].set_axisbelow(True)
#         axs[i, j].legend(handles[:0], labels[:0])
        if i==0 and j==3:
            continue
        else:
            axs[i,j].get_legend().remove()
    
sns.move_legend(axs[0,3], "upper left", bbox_to_anchor=(1, 1), )
sns.despine()
# axs.set(axs, xlabel=None)

fig.tight_layout()

foname = (output_imgdir+"/MCMC_modelparam_boxplot_{}.png".format(freqband))
plt.savefig(foname, dpi=300, bbox_inches='tight')

foname = (output_imgdir+"/MCMC_modelparam_boxplot_{}.eps".format(freqband))
plt.savefig(foname, dpi=300, bbox_inches='tight')

# Plot paramters as a function of fault normal distance

Run the `Maps/BPnetwork_Faultdist/code/BPnetwork_computeFaultdist.ipynb` to compute the fault normal distance and copy the data into the `../data_stats/`.

In [ ]:
df_stationdist = pd.read_csv("../data_stats/BPnetwork_faultnormaldist.csv", index_col=0)

In [ ]:
df_stationdist

In [ ]:
# # append pairtype: Pacific, North America, Crossing
# for index, row in df_all_residuthresh.iterrows():
#     print(index, )
    

In [ ]:
# index = "BP.LCCB-BP.MMNB"
for index, row in df_all_residuthresh.iterrows():

    netsta1, netsta2 = index.split("-")
    sta1, sta2 = [x.split(".")[1] for x in index.split("-")]
    # find corrtype
    if sta1 == sta2:
        corrtype = "AC" # auto-correlation
    else:
        corrtype = "CC" # cross-correlation

    # find loctype
    staside1 = df_stationdist.loc[sta1].locside
    staside2 = df_stationdist.loc[sta2].locside

    if staside1==staside2:
        if staside1=="northamerican":
            loctype = "North America"
        elif staside1=="pacific":
            loctype = "Pacific"
            
    else:
        loctype = "Crossing"

    loc_corrtype = f"{loctype}-{corrtype}"

    # Fault normal distance
    stadist1 = df_stationdist.loc[sta1].faultnormal_dist
    stadist2 = df_stationdist.loc[sta2].faultnormal_dist
    pairdist = np.mean([stadist1, stadist2]) # fault normal distance is evaluated as an average of two stations; in the case of AC stadist1 and stadist1 are identical.
    df_all_residuthresh.loc[index, "pairdist"] = pairdist
    df_all_residuthresh.loc[index, "loc_corrtype"] = loc_corrtype



In [ ]:
df_all_residuthresh

In [ ]:
# Extract the model case of with trend and divide into stretching and mwcs
df_all_residuthresh_base = df_all_residuthresh[df_all_residuthresh["modelcase"]=="base"]
df_all_residuthresh_wlin = df_all_residuthresh[df_all_residuthresh["modelcase"]=="w. linear trend"]

df_all_residuthresh_wlin_stretching = df_all_residuthresh_wlin[df_all_residuthresh_wlin["method"]=="stretching"]
df_all_residuthresh_wlin_mwcs = df_all_residuthresh_wlin[df_all_residuthresh_wlin["method"]=="MWCS"]

In [ ]:
df_annot_all = pd.DataFrame(columns=["method", "index", "x", "y", "xtext", "ytext"])

# Assemble for the annotations of outliers
annotation_index_stretching = ['BP.GHIB-BP.GHIB', 'BP.LCCB-BP.VCAB']
annotation_index_mwcs = ['BP.EADB-BP.GHIB', 'BP.EADB-BP.MMNB', 'BP.LCCB-BP.LCCB']

dxytext_stretching = [(0.5, -0.001), (0.5, 0.001)]
dxytext_mwcs= [(0.4, 0.0), (0.5, 6e-4), (0.5, -8e-4)]

for i, annotindex in enumerate(annotation_index_stretching):
#     annotindex = annotation_index_stretching[0]
    sta1, sta2 = [x.split(".")[1] for x in annotindex.split("-")]
    df_annot = df_all_residuthresh_wlin_stretching[df_all_residuthresh_wlin_stretching.index == annotindex]
    annot_blin = df_annot.blin_1overyear.values[0]
    annot_dist = df_annot.pairdist.values[0]
    d_annot = {"method":["stretching"], "index":f"{sta1}-{sta2}", "x":[annot_dist], "y":[annot_blin], 
               "xtext":[annot_dist+dxytext_stretching[i][0]], "ytext":[annot_blin++dxytext_stretching[i][1]]}
    df_annot_all = pd.concat([df_annot_all, pd.DataFrame.from_dict(d_annot)], ignore_index=True)

for i, annotindex in enumerate(annotation_index_mwcs):
#     annotindex = annotation_index_stretching[0]
    sta1, sta2 = [x.split(".")[1] for x in annotindex.split("-")]
    df_annot = df_all_residuthresh_wlin_mwcs[df_all_residuthresh_wlin_mwcs.index == annotindex]
    annot_blin = df_annot.blin_1overyear.values[0]
    annot_dist = df_annot.pairdist.values[0]
    d_annot = {"method":["MWCS"], "index":f"{sta1}-{sta2}", "x":[annot_dist], "y":[annot_blin], 
               "xtext":[annot_dist+dxytext_mwcs[i][0]], "ytext":[annot_blin++dxytext_mwcs[i][1]]}
    df_annot_all = pd.concat([df_annot_all, pd.DataFrame.from_dict(d_annot)], ignore_index=True)

df_annot_all

## Plot b_{lin} with fault normal distance 

In [ ]:
np.random.seed(20230419)
cpalette_scatter="colorblind"

plt.rcParams["xtick.minor.visible"] = True
fig, axs = plt.subplots(1, 2, figsize=(10, 4))

# axs[0].grid(True, which="both")
# axs[1].grid(True, which="both")

key = "blin_1overyear" #"b_{lin}"
ms = 50

hue_order=["Pacific-AC", "Pacific-CC", "North America-AC", "North America-CC", "Crossing-CC"]

sns.scatterplot(x="pairdist", y=key, data=df_all_residuthresh_wlin_stretching, hue="loc_corrtype", style="loc_corrtype", hue_order=hue_order,
                palette=cpalette_scatter, s=ms, edgecolor="w", linewidth=0.5, ax=axs[0])
sns.scatterplot(x="pairdist", y=key, data=df_all_residuthresh_wlin_mwcs, hue="loc_corrtype", style="loc_corrtype", hue_order=hue_order,
                palette=cpalette_scatter, s=ms, edgecolor="w", linewidth=0.5, ax=axs[1])

axs[0].set_title("stretching")
axs[1].set_title("MWCS")

axs[0].set_ylim([-0.002, 0.014])
axs[1].set_ylim([-0.002, 0.014])

# plt.setp(ax, xlabel=None)
# ax.legend()
axs[0].get_legend().remove()
sns.move_legend(axs[1], "upper left", bbox_to_anchor=(1, 1), title="")
# sns.despine()

for i in range(2):
    axs[i].set_xlim([0, 5])
    axs[i].set_xlabel("Averaged fault normal distance [km]")
    axs[i].set_ylabel("Slope $b_0$ [%/year]")
#     axs[i].set_yscale("log")

arrow_dict = dict(color="black", shrink=0.2, width=0, headwidth=0 )
                  
#---Annotate index for some outliers---#
for index, row in df_annot_all.iterrows():
    if row["method"] == "stretching":
        figid = 0
    else:
        figid = 1

    axs[figid].annotate(row["index"],
                xy = (row.x, row.y), xytext = (row.xtext, row.ytext),
                arrowprops = arrow_dict, size = 11, horizontalalignment='left',)

fig.tight_layout()

foname = (output_imgdir+"/MCMC_modelparam_blin_withFaultnormaldist_{}.png".format(freqband))
plt.savefig(foname, dpi=300, bbox_inches='tight')

foname = (output_imgdir+"/MCMC_modelparam_blin_withFaultnormaldist_{}.eps".format(freqband))
plt.savefig(foname, dpi=300, bbox_inches='tight')

# Plot the other model parameters with fault normal distance

We plot the results associated with base & w.lin, and stretching and mwcs separately. 

## plot base model

In [ ]:
dvvmodel = "stretching"
df_all_residuthresh_base[df_all_residuthresh_base["method"]==dvvmodel].columns

In [ ]:
np.random.seed(20230427)
fig, axs = plt.subplots(6, 4, figsize=(14, 16), sharex=False, gridspec_kw=dict(height_ratios=[1, 1, 1, 0.02, 1, 1]))

for i in range(4):
    axs[0, i].set_axis_off()
    axs[3, i].set_axis_off()

ms = 50
hue_order=["Pacific-AC", "Pacific-CC", "North America-AC", "North America-CC", "Crossing-CC"]

for ii, dvvmodel in enumerate(["stretching", "MWCS"]):
    if ii == 0:
        i1, i2 = (1, 2)
    else:
        i1, i2 = (4, 5)
    
    df_all_residuthresh_base_case = df_all_residuthresh_base[df_all_residuthresh_base["method"]==dvvmodel]
    
    # a0
    key="a0"
    sns.scatterplot(x="pairdist", y=key, data=df_all_residuthresh_base_case, hue="loc_corrtype", style="loc_corrtype", hue_order=hue_order,
            palette=cpalette_scatter, s=ms, edgecolor="w", linewidth=0.5, ax=axs[i1, 0])
    axs[i1, 0].set_ylabel("$a_0$")
    axs[i1, 0].set_ylim([-0.1, 0.05])

    # p1
    key="p1"
    sns.scatterplot(x="pairdist", y=key, data=df_all_residuthresh_base_case, hue="loc_corrtype", style="loc_corrtype", hue_order=hue_order,
        palette=cpalette_scatter, s=ms, edgecolor="w", linewidth=0.5, ax=axs[i1, 1])
    axs[i1, 1].set_ylabel("$p_1$")
    axs[i1, 1].set_ylim([-0.01, 0.001])
    
    # p2
    key="p2"
    sns.scatterplot(x="pairdist", y=key, data=df_all_residuthresh_base_case, hue="loc_corrtype", style="loc_corrtype", hue_order=hue_order,
        palette=cpalette_scatter, s=ms, edgecolor="w", linewidth=0.5, ax=axs[i1, 2])
    axs[i1, 2].set_ylabel("$p_2$")
    axs[i1, 2].set_ylim([-0.0022, 0.0022])
    
    # t_shift
    key="t_{shiftdays}"
    sns.scatterplot(x="pairdist", y=key, data=df_all_residuthresh_base_case, hue="loc_corrtype", style="loc_corrtype", hue_order=hue_order,
        palette=cpalette_scatter, s=ms, edgecolor="w", linewidth=0.5, ax=axs[i1, 3])
    axs[i1, 3].set_ylabel(r'$t_{0}^{shift}$ [days]')
    axs[i1, 3].set_ylim([-5, 95])
    axs[i1, 3].set_yticks(np.arange(0, 95, 15))
    
    Slimit = [-0.002, 0.1]
    tmaxlimit=[1e-1, 1e4]

    # S1
    key="S1"
    sns.scatterplot(x="pairdist", y=key, data=df_all_residuthresh_base_case, hue="loc_corrtype", style="loc_corrtype", hue_order=hue_order,
        palette=cpalette_scatter, s=ms, edgecolor="w", linewidth=0.5, ax=axs[i2, 0])
    axs[i2, 0].set_ylabel("$s_1$")
    axs[i2, 0].set_ylim(Slimit)


    # tmax1
    key="tmax1"
    sns.scatterplot(x="pairdist", y=key, data=df_all_residuthresh_base_case, hue="loc_corrtype", style="loc_corrtype", hue_order=hue_order,
        palette=cpalette_scatter, s=ms, edgecolor="w", linewidth=0.5, ax=axs[i2, 1])
    axs[i2, 1].set_ylabel(r"$\tau_{1}^{max}$ [year]")
    axs[i2, 1].set_yscale('log')
    axs[i2, 1].set_ylim(tmaxlimit)

    # S2
    key="S2"
    sns.scatterplot(x="pairdist", y=key, data=df_all_residuthresh_base_case, hue="loc_corrtype", style="loc_corrtype", hue_order=hue_order,
        palette=cpalette_scatter, s=ms, edgecolor="w", linewidth=0.5, ax=axs[i2, 2])
    axs[i2, 2].set_ylabel("$s_2$")
    axs[i2, 2].set_ylim(Slimit)

    # tmax2
    key="tmax2"
    sns.scatterplot(x="pairdist", y=key, data=df_all_residuthresh_base_case, hue="loc_corrtype", style="loc_corrtype", hue_order=hue_order,
        palette=cpalette_scatter, s=ms, edgecolor="w", linewidth=0.5, ax=axs[i2, 3])
    axs[i2, 3].set_ylabel(r"$\tau_{2}^{max}$ [year]")
    axs[i2, 3].set_yscale('log')
    axs[i2, 3].set_ylim(tmaxlimit)
    

plt.setp(axs, xlabel=None)

for i in [1, 2, 4, 5]:
    for j in range(4):
        axs[i, j].set_xlim([0, 5])
        axs[i, j].set_xticks(range(6))
        axs[i, j].set_xlabel("Averaged fault normal distance [km]")
#         axs[i, j].grid(True, which='both', axis='both')
        axs[i, j].set_axisbelow(False)
        if i==1 and j==3:
            continue
        else:
            axs[i,j].get_legend().remove()
    
fig.tight_layout()

sns.move_legend(axs[1, 3], "upper center", bbox_to_anchor=(0.6, 1.7), title="")

plt.text(0, 0.2, 'Base model', horizontalalignment='center', verticalalignment='center', transform = axs[0,0].transAxes, fontsize=16, fontweight="bold")
plt.text(0, 0, '(a) Stretching', horizontalalignment='center', verticalalignment='center', transform = axs[0,0].transAxes, fontsize=16, fontweight="bold")
plt.text(0, -6.0, '(b) MWCS', horizontalalignment='center', verticalalignment='center', transform = axs[3,0].transAxes, fontsize=16, fontweight="bold")

foname = (output_imgdir+"/MCMC_modelparam_all_withFaultnormaldist_base_{}.png".format(freqband))
plt.savefig(foname, dpi=300, bbox_inches='tight')

## Plot w.lin model

In [ ]:
np.random.seed(20230427)
fig, axs = plt.subplots(6, 4, figsize=(14, 16), sharex=False, gridspec_kw=dict(height_ratios=[1, 1, 1, 0.02, 1, 1]))

for i in range(4):
    axs[0, i].set_axis_off()
    axs[3, i].set_axis_off()

ms = 50
hue_order=["Pacific-AC", "Pacific-CC", "North America-AC", "North America-CC", "Crossing-CC"]

for ii, dvvmodel in enumerate(["stretching", "MWCS"]):
    if ii == 0:
        i1, i2 = (1, 2)
    else:
        i1, i2 = (4, 5)
    
    df_all_residuthresh_wlin_case = df_all_residuthresh_wlin[df_all_residuthresh_wlin["method"]==dvvmodel]
    
    # a0
    key="a0"
    sns.scatterplot(x="pairdist", y=key, data=df_all_residuthresh_wlin_case, hue="loc_corrtype", style="loc_corrtype", hue_order=hue_order,
            palette=cpalette_scatter, s=ms, edgecolor="w", linewidth=0.5, ax=axs[i1, 0])
    axs[i1, 0].set_ylabel("$a_0$")
    axs[i1, 0].set_ylim([-0.1, 0.05])

    # p1
    key="p1"
    sns.scatterplot(x="pairdist", y=key, data=df_all_residuthresh_wlin_case, hue="loc_corrtype", style="loc_corrtype", hue_order=hue_order,
        palette=cpalette_scatter, s=ms, edgecolor="w", linewidth=0.5, ax=axs[i1, 1])
    axs[i1, 1].set_ylabel("$p_1$")
    axs[i1, 1].set_ylim([-0.01, 0.001])
    
    # p2
    key="p2"
    sns.scatterplot(x="pairdist", y=key, data=df_all_residuthresh_wlin_case, hue="loc_corrtype", style="loc_corrtype", hue_order=hue_order,
        palette=cpalette_scatter, s=ms, edgecolor="w", linewidth=0.5, ax=axs[i1, 2])
    axs[i1, 2].set_ylabel("$p_2$")
    axs[i1, 2].set_ylim([-0.0022, 0.0022])
    
    # t_shift
    key="t_{shiftdays}"
    sns.scatterplot(x="pairdist", y=key, data=df_all_residuthresh_wlin_case, hue="loc_corrtype", style="loc_corrtype", hue_order=hue_order,
        palette=cpalette_scatter, s=ms, edgecolor="w", linewidth=0.5, ax=axs[i1, 3])
    axs[i1, 3].set_ylabel(r"$t_{0}^{shift}$ [days]")
    axs[i1, 3].set_ylim([-5, 95])
    axs[i1, 3].set_yticks(np.arange(0, 95, 15))
    
    Slimit = [-0.002, 0.1]
    tmaxlimit=[1e-1, 1e4]

    # S1
    key="S1"
    sns.scatterplot(x="pairdist", y=key, data=df_all_residuthresh_wlin_case, hue="loc_corrtype", style="loc_corrtype", hue_order=hue_order,
        palette=cpalette_scatter, s=ms, edgecolor="w", linewidth=0.5, ax=axs[i2, 0])
    axs[i2, 0].set_ylabel("$s_1$")
    axs[i2, 0].set_ylim(Slimit)


    # tmax1
    key="tmax1"
    sns.scatterplot(x="pairdist", y=key, data=df_all_residuthresh_wlin_case, hue="loc_corrtype", style="loc_corrtype", hue_order=hue_order,
        palette=cpalette_scatter, s=ms, edgecolor="w", linewidth=0.5, ax=axs[i2, 1])
    axs[i2, 1].set_ylabel(r"$\tau_{1}^{max}$ [year]")
    axs[i2, 1].set_yscale('log')
    axs[i2, 1].set_ylim(tmaxlimit)

    # S2
    key="S2"
    sns.scatterplot(x="pairdist", y=key, data=df_all_residuthresh_wlin_case, hue="loc_corrtype", style="loc_corrtype", hue_order=hue_order,
        palette=cpalette_scatter, s=ms, edgecolor="w", linewidth=0.5, ax=axs[i2, 2])
    axs[i2, 2].set_ylabel("$s_2$")
    axs[i2, 2].set_ylim(Slimit)

    # tmax2
    key="tmax2"
    sns.scatterplot(x="pairdist", y=key, data=df_all_residuthresh_wlin_case, hue="loc_corrtype", style="loc_corrtype", hue_order=hue_order,
        palette=cpalette_scatter, s=ms, edgecolor="w", linewidth=0.5, ax=axs[i2, 3])
    axs[i2, 3].set_ylabel(r"$\tau_{2}^{max}$ [year]")
    axs[i2, 3].set_yscale('log')
    axs[i2, 3].set_ylim(tmaxlimit)
    

plt.setp(axs, xlabel=None)

for i in [1, 2, 4, 5]:
    for j in range(4):
        axs[i, j].set_xlim([0, 5])
        axs[i, j].set_xticks(range(6))
        axs[i, j].set_xlabel("Averaged fault normal distance [km]")
#         axs[i, j].grid(True, which='both', axis='both')
        axs[i, j].set_axisbelow(False)
        if i==1 and j==3:
            continue
        else:
            axs[i,j].get_legend().remove()
    
fig.tight_layout()

sns.move_legend(axs[1, 3], "upper center", bbox_to_anchor=(0.6, 1.7), title="")

plt.text(0, 0.2, 'w. linear trend', horizontalalignment='center', verticalalignment='center', transform = axs[0,0].transAxes, fontsize=16, fontweight="bold")
plt.text(0, 0, '(a) Stretching', horizontalalignment='center', verticalalignment='center', transform = axs[0,0].transAxes, fontsize=16, fontweight="bold")
plt.text(0, -6.0, '(b) MWCS', horizontalalignment='center', verticalalignment='center', transform = axs[3,0].transAxes, fontsize=16, fontweight="bold")

foname = (output_imgdir+"/MCMC_modelparam_all_withFaultnormaldist_wlin_{}.png".format(freqband))
plt.savefig(foname, dpi=300, bbox_inches='tight')

In [ ]:
# save the sorted station list to plot with the sorted order by fault normal distance
for dvvmethod in ["stretching", "mwcs"]:
    stationpairs_dict_wlin = dict(station_all=df_all_residuthresh_wlin.loc[df_all_residuthresh_wlin["method"]==dvvmethod].sort_values(by=['pairdist']).index)
    pairsdist_dict_wlin = dict(pairdist=df_all_residuthresh_wlin.loc[df_all_residuthresh_wlin["method"]==dvvmethod].sort_values(by=['pairdist']).pairdist)

    with open(inputdatadir+f'/plot_dvvmodel_sortedbydist_wlin_{freqband}_{dvvmethod}.pickle', 'wb') as fo:
        pickle.dump(stationpairs_dict_wlin, fo)
        pickle.dump(pairsdist_dict_wlin, fo)

# Plot the coseismic drop with fault normal distance

The coseismic drop of dv/v with log healing model is given as follows (Snieder et al. 2017):

$$ SR(t_0) = S \log \left(\dfrac{\tau_{max}}{\tau_{min}} \right). $$

It is noted that the coefficient S does not directly represent the coseismic decrease in dv/v. We thus plot it as a function of fault normal distance.

In [ ]:
df_all_residuthresh_wlin_case.columns

In [ ]:
for ii, dvvmodel in enumerate(["stretching", "mwcs"]):
    if ii == 0:
        i1, i2 = (1, 2)
    else:
        i1, i2 = (4, 5)
    
    df_all_residuthresh_wlin_case = df_all_residuthresh_wlin[df_all_residuthresh_wlin["method"]==dvvmodel]
    

In [ ]:
def get_coseismicdvvdrop(S, logtaumin, logtaumax):
    # rescale taumin and taumax
    taumin = 10**logtaumin # [s]
    taumax = 10**logtaumax # [s]
    return -S*np.log(taumax/taumin) # return as negative



In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 3), sharex=False,)

dvvdroplimit=[-0.2, 0]
cpalette_scatter = "colorblind"
ms = 50
hue_order=["Pacific-AC", "Pacific-CC", "North America-AC", "North America-CC", "Crossing-CC"]

for ii, dvvmodel in enumerate(["stretching", "MWCS"]):
    df_all_residuthresh_wlin_case = df_all_residuthresh_wlin[df_all_residuthresh_wlin["method"]==dvvmodel].copy()
    # Parkfield event corresponds to index 2
    df_all_residuthresh_wlin_case.loc[:, "coseismicdvvdrop"] = df_all_residuthresh_wlin_case.apply(lambda x: get_coseismicdvvdrop(x.S2, x.log10tmin2, x.log10tmax2), axis=1)

    key="coseismicdvvdrop"
    sns.scatterplot(x="pairdist", y=key, data=df_all_residuthresh_wlin_case, hue="loc_corrtype", style="loc_corrtype", hue_order=hue_order,
        palette=cpalette_scatter, s=ms, edgecolor="w", linewidth=0.5, ax=axs[ii])
    axs[ii].set_ylabel("coseismic dv/v decrease\nS$^{PF}$log(τmax/τmin)[%]")
    axs[ii].set_ylim(dvvdroplimit)    

plt.setp(axs, xlabel=None)

for i in range(2):
    axs[i].set_xlim([0, 5])
    axs[i].set_xticks(range(6))
    axs[i].set_xlabel("Averaged fault normal distance [km]")
#         axs[i].grid(True, which='both', axis='both')
    axs[i].set_axisbelow(False)

axs[0].get_legend().remove()
          
axs[0].set_title("stretching")
axs[1].set_title("MWCS")

fig.tight_layout()

sns.move_legend(axs[1], "upper center", bbox_to_anchor=(1.4, 1.04), title="")

# plt.text(0, 0.2, 'w. linear trend', horizontalalignment='center', verticalalignment='center', transform = axs[0,0].transAxes, fontsize=16, fontweight="bold")
# plt.text(0, 0, '(a) Stretching', horizontalalignment='center', verticalalignment='center', transform = axs[0,0].transAxes, fontsize=16, fontweight="bold")
# plt.text(0, -6.0, '(b) mwcs', horizontalalignment='center', verticalalignment='center', transform = axs[3,0].transAxes, fontsize=16, fontweight="bold")

foname = (output_imgdir+"/MCMC_modelparam_coseismicdrop_withFaultnormaldist_wlin_{}.png".format(freqband))
plt.savefig(foname, dpi=300, bbox_inches='tight')

foname = (output_imgdir+"/MCMC_modelparam_coseismicdrop_withFaultnormaldist_wlin_{}.eps".format(freqband))
plt.savefig(foname, dpi=300, bbox_inches='tight')

# Conclusion

In this notebook we plotted the stastical analysis of model parameters estimated by the MCMC inversion.